In [1]:
# import libraries
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import inspect
import time

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import src.simulation as s

%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

In [2]:
try:
    input_args = json.loads(open('../input/simulation.json').read())
except Exception as ex:
    print('simulation.json does not exist!')
    print(ex)


In [12]:
# instantiate simulation class

simulation =  s.Simulation()

/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:29: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.PeerNominatedDataPopulation = p.PeerNominatedDataPopulation('Peer-Nominated data population', self.input_args)
<ipython-input-12-6c4390bb5dac>:3: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  simulation =  s.Simulation()
/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:30: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.CommunicationDataPopulation = p.CommunicationDataPopulation('Communication data population', self.input_args)


# Simulation Interventions

## 1. Nomination network

In [5]:
# Read tuned parameter combinations (See tuning_nom_network.ipynb)
pars_nomination = pd.read_csv('../output/opt_pars_nomination.csv', sep=',', header=0)
pars_nomination

,threshold,ipa,error
0,0.025,0.007,0.462071
1,0.025,0.004,0.471020
2,0.025,0.005,0.471045
3,0.025,0.006,0.471688
4,0.025,0.009,0.472211
...,...,...,...
95,0.950,0.005,0.682455
96,1.000,0.045,0.682522
97,1.000,0.038,0.682627
98,1.000,0.042,0.682804


In [13]:
# Run simulations for each parameter combination
list_results_nomm = []
list_results_avg_nomm = []
list_agents_per_intervention_nomm = []
start_whole = time.time()
count = 0
for index, row in pars_nomination.iterrows():
    results_nomm, results_avg_nomm, agents_per_intervention_nomm = simulation.simulate_interventions(700,
                                                                                                     'nomination',
                                                                                                     row['threshold'],
                                                                                                     row['ipa'])
    list_results_nomm.append(results_nomm)
    list_results_avg_nomm.append(results_avg_nomm)
    list_agents_per_intervention_nomm.append(agents_per_intervention_nomm)
    end = time.time()
    print(count, row['threshold'], row['ipa'], "Time elapsed:", end - start_whole, ' seconds')
    count = count + 1


0 0.025 0.006999999999999999 Time elapsed: 23.890190839767456  seconds
1 0.025 0.004 Time elapsed: 47.6353280544281  seconds
2 0.025 0.005 Time elapsed: 70.87401509284973  seconds
3 0.025 0.006 Time elapsed: 93.93312811851501  seconds
4 0.025 0.009000000000000001 Time elapsed: 115.90274000167847  seconds
5 0.025 0.003 Time elapsed: 138.48475193977356  seconds
6 0.025 0.008 Time elapsed: 160.37360000610352  seconds
7 0.025 0.012 Time elapsed: 183.01990413665771  seconds
8 0.025 0.010000000000000002 Time elapsed: 205.21095991134644  seconds
9 0.025 0.011000000000000001 Time elapsed: 227.35793209075928  seconds
10 0.025 0.002 Time elapsed: 249.39573907852173  seconds
11 0.025 0.013 Time elapsed: 271.21685314178467  seconds
12 0.025 0.014000000000000002 Time elapsed: 293.1421639919281  seconds
13 0.025 0.015 Time elapsed: 315.41246700286865  seconds
14 0.05 0.001 Time elapsed: 337.48151206970215  seconds
15 0.025 0.001 Time elapsed: 359.1723909378052  seconds
16 0.07500000000000001 0.001 T

## 2. Communication network


In [14]:
# Read tuned parameter combinations (See tuning_com_network.ipynb)
pars_communication = pd.read_csv('../output/opt_pars_communication.csv', sep=',', header=0, encoding='latin-1')
pars_communication

,threshold,ipa,error
0,1.000,0.001,0.675750
1,0.975,0.001,0.683444
2,0.950,0.001,0.691740
3,0.925,0.001,0.700577
4,0.900,0.001,0.709325
...,...,...,...
95,0.950,0.013,0.744159
96,0.975,0.032,0.744769
97,0.950,0.018,0.744785
98,1.000,0.048,0.745074


In [16]:
# Run simulations for each parameter combination
list_results_comm = []
list_results_avg_comm = []
list_agents_per_intervention_comm = []
start_whole = time.time()
count = 0
for index, row in pars_communication.iterrows():
    results_comm, results_avg_comm, agents_per_intervention_comm = simulation.simulate_interventions(700,'communication',row['threshold'],row['ipa'])
    list_results_comm.append(results_comm)
    list_results_avg_comm.append(results_avg_comm)
    list_agents_per_intervention_comm.append(agents_per_intervention_comm)
    end = time.time()
    print(count, row['threshold'], row['ipa'], "Time elapsed:", end - start_whole, ' seconds')
    count = count + 1

FileCreateError: [Errno 2] No such file or directory: '../output/selectedAgents/selected_agents_communication_1.0_0.001.xlsx'

# Ploting the outcomes

## Postprocessing output

In [ ]:
# Step1: create a list per run of overall mean PAL

# Nomination network
list_results_mean_nom = []
for run in range(len(list_results_avg_nomm)):
    # per run mean of all classes
    all_averaged = {}
    for i in input_args['intervention_strategy']:
        temp_res = pd.Series([], dtype = float)
        counter = 0
        for class_id,res in list_results_avg_nomm[run].items():
            temp_res = temp_res.add(list_results_avg_nomm[run][class_id][i],fill_value=0)
            counter = counter + 1
        all_averaged[i] = temp_res/counter

    list_results_mean_nom.append(all_averaged)



# Communication network
list_results_mean_com = []
for run in range(len(list_results_avg_comm)):
    # per run mean of all classes
    all_averaged = {}
    for i in input_args['intervention_strategy']:
        temp_res = pd.Series([], dtype = float)
        counter = 0
        for class_id,res in list_results_avg_comm[run].items():
            temp_res = temp_res.add(list_results_avg_comm[run][class_id][i],fill_value=0)
            counter = counter + 1
        all_averaged[i] = temp_res/counter

    list_results_mean_com.append(all_averaged)


In [ ]:
# Step2: Creat per intervention a list of 100

# Nomination network
out_indegree_nom = []
out_betweenness_nom = []
out_closeness_nom = []

for run in range(len(list_results_mean_nom)):
    # per run
    temp1 = list_results_mean_nom[run]
    out_indegree_nom.append(temp1['indegree'])
    out_betweenness_nom.append(temp1['betweenness'])
    out_closeness_nom.append(temp1['closeness'])


# Comunication network
out_indegree_com = []
out_betweenness_com = []
out_closeness_com = []

for run in range(len(list_results_mean_com)):
    # per run
    temp1 = list_results_mean_com[run]
    out_indegree_com.append(temp1['indegree'])
    out_betweenness_com.append(temp1['betweenness'])
    out_closeness_com.append(temp1['closeness'])


In [ ]:
# Step3: calculate mean and percentiles

# list into dataframe

# Nomination network
results_nom_indegree = pd.concat(out_indegree_nom, axis=1, keys=[s.name for s in out_indegree_nom])
results_nom_betweenness = pd.concat(out_betweenness_nom, axis=1, keys=[s.name for s in out_betweenness_nom])
results_nom_closeness = pd.concat(out_closeness_nom, axis=1, keys=[s.name for s in out_closeness_nom])

# indegree
i_nom = results_nom_indegree.mean(axis=1)
i_nom_min = results_nom_indegree.quantile(q=0.025, axis=1)
i_nom_max = results_nom_indegree.quantile(q=0.975, axis=1)

# betweenness
b_nom = results_nom_betweenness.mean(axis=1)
b_nom_min = results_nom_betweenness.quantile(q=0.025, axis=1)
b_nom_max = results_nom_betweenness.quantile(q=0.975, axis=1)

# closeness
c_nom = results_nom_closeness.mean(axis=1)
c_nom_min = results_nom_closeness.quantile(q=0.025, axis=1)
c_nom_max = results_nom_closeness.quantile(q=0.975, axis=1)


# Comunication network
results_com_indegree = pd.concat(out_indegree_com, axis=1, keys=[s.name for s in out_indegree_com])
results_com_betweenness = pd.concat(out_betweenness_com, axis=1, keys=[s.name for s in out_betweenness_com])
results_com_closeness = pd.concat(out_closeness_com, axis=1, keys=[s.name for s in out_closeness_com])

# indegree
i_com = results_com_indegree.mean(axis=1)
i_com_min = results_com_indegree.quantile(q=0.025, axis=1)
i_com_max = results_com_indegree.quantile(q=0.975, axis=1)

# betweenness
b_com = results_com_betweenness.mean(axis=1)
b_com_min = results_com_betweenness.quantile(q=0.025, axis=1)
b_com_max = results_com_betweenness.quantile(q=0.975, axis=1)

# closeness
c_com = results_com_closeness.mean(axis=1)
c_com_min = results_com_closeness.quantile(q=0.025, axis=1)
c_com_max = results_com_closeness.quantile(q=0.975, axis=1)

In [ ]:
# Success rate (only mean)

# Nomination network
i_nom_sr = (i_nom/i_nom[0] -1) *100
i_nom_sr_min = (i_nom_min/i_nom_min[0] -1) *100
i_nom_sr_max = (i_nom_max/i_nom_max[0] -1) *100
b_nom_sr = (b_nom/b_nom[0] -1) *100
b_nom_sr_min = (b_nom_min/b_nom_min[0] -1) *100
b_nom_sr_max = (b_nom_max/b_nom_max[0] -1) *100
c_nom_sr = (c_nom/c_nom[0] -1) *100
c_nom_sr_min = (c_nom_min/c_nom_min[0] -1) *100
c_nom_sr_max = (c_nom_max/c_nom_max[0] -1) *100

# Communication network
i_com_sr = (i_com/i_com[0] -1) *100
i_com_sr_min = (i_com_min/i_com_min[0] -1) *100
i_com_sr_max = (i_com_max/i_com_max[0] -1) *100
b_com_sr = (b_com/b_com[0] -1) *100
b_com_sr_min = (b_com_min/b_com_min[0] -1) *100
b_com_sr_max = (b_com_max/b_com_max[0] -1) *100
c_com_sr = (c_com/c_com[0] -1) *100
c_com_sr_min = (c_com_min/c_com_min[0] -1) *100
c_com_sr_max = (c_com_max/c_com_max[0] -1) *100


In [ ]:
# Plots (with confidence intervals)
ticks = np.arange(1.40, 1.60, 0.01)
x = np.arange(0,700,1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,10))

ax1.set_title('Nomination network')
ax1.set_ylim([1.4, 1.55])
ax1.plot(x, i_nom, color='green')
#plt.fill_between(x, i_nom_min, i_nom_max, alpha=0.2, color='C5')
ax1.plot(x, b_nom, color='blue')
#plt.fill_between(x, b_nom_min, b_nom_max, alpha=0.2, color='C0')
ax1.plot(x, c_nom, color='orange')
#plt.fill_between(x, c_nom_min, c_nom_max, alpha=0.2, color='C3')

ax2.set_title('Communication network')
ax2.set_ylim([1.4, 1.55])
ax2.plot(x, i_com, color='green')
#plt.fill_between(x, i_com_min, i_com_max, alpha=0.2, color='C5')
ax2.plot(x, b_com, color='blue')
#plt.fill_between(x, b_com_min, b_com_max, alpha=0.2, color='C0')
ax2.plot(x, c_com, color='orange')
#plt.fill_between(x, c_com_min, c_com_max, alpha=0.2, color='C3')
ax2.legend(['In-degree', 'Betweenness', 'Closeness'], loc='lower right', title="Network interventions")

#fig.figsize=((16,14))
#plt.axis(xmin=0,xmax=700,ymin=1.4,ymax=1.6)
#plt.yticks(ticks)
#plt.ylabel('Mean physical activity level')
#plt.xlabel('Days')
#plt.legend(['In-Degree', 'Betweenness', 'Closeness'], loc='lower right', title="Network interventions")

fig.savefig('../output/sim_results1.eps', bbox_inches='tight', format='eps', dpi=1000)


In [ ]:
# confidence intervals at day 700
print('nom i:', i_nom_sr[699], 'b:', b_nom_sr[699], 'c: ', c_nom_sr[699])
print('nom min i:', i_nom_sr_min[699], 'b:', b_nom_sr_min[699], 'c: ', c_nom_sr_min[699])
print('nom max i:', i_nom_sr_max[699], 'b:', b_nom_sr_max[699], 'c: ', c_nom_sr_max[699])

print('com i:', i_com_sr[699], 'b:', b_com_sr[699], 'c: ', c_com_sr[699])
print('com min i:', i_com_sr_min[699], 'b:', b_com_sr_min[699], 'c: ', c_com_sr_min[699])
print('com max i:', i_com_sr_max[699], 'b:', b_com_sr_max[699], 'c: ', c_com_sr_max[699])

In [ ]:
c_com_sr

In [ ]:
# Plots (without confidence intervals)
ticks = np.arange(1.40, 1.60, 0.01)
x = np.arange(0,700,1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,7))

ax1.set_title('Nomination network')
ax1.set_xlabel('Days')
ax1.set_ylabel('Impact on physical activity level (%)')
ax1.set_ylim([0, 10])
ax1.plot(x, i_nom_sr, color='green')
ax1.plot(x, b_nom_sr, color='blue')
ax1.plot(x, c_nom_sr, color='orange')

ax2.set_title('Communication network')
ax2.set_xlabel('Days')
ax2.set_ylabel('Impact on physical activity level (%)')
ax2.set_ylim([0, 10])
ax2.plot(x, i_com_sr, color='green')
ax2.plot(x, b_com_sr, color='blue')
ax2.plot(x, c_com_sr, color='orange')
ax2.legend(['In-degree', 'Betweenness', 'Closeness'], loc='lower right', title="Network interventions")

fig.savefig('../output/sim_results2.eps', bbox_inches='tight', format='eps', dpi=1000)


# Variation at day 700

In [ ]:
nom_var = {'In-degree': (results_nom_indegree.iloc[699]/results_nom_indegree.iloc[0] -1) *100,
           'Betweenness': (results_nom_betweenness.iloc[699]/results_nom_betweenness.iloc[0] -1) *100,
           'Closeness': (results_nom_closeness.iloc[699]/results_nom_closeness.iloc[0] -1) *100}

com_var = {'In-degree': (results_com_indegree.iloc[699]/results_com_indegree.iloc[0] -1) *100,
           'Betweenness': (results_com_betweenness.iloc[699]/results_com_betweenness.iloc[0] -1) *100,
           'Closeness': (results_com_closeness.iloc[699]/results_com_closeness.iloc[0] -1) *100}


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,10))

ax1.set_title('Nomination network')
ax1.set_ylabel('PAL success rate (%)')
ax1.set_ylim([0, 10])
ax1.boxplot(nom_var.values())
ax1.set_xticklabels(nom_var.keys())


ax2.set_title('Communication network')
ax2.set_ylabel('PAL success rate (%)')
ax2.set_ylim([0, 10])
ax2.boxplot(com_var.values())
ax2.set_xticklabels(com_var.keys())
#ax2.legend(['In-degree', 'Betweenness', 'Closeness'], loc='lower right', title="Network interventions")

fig.savefig('../output/sim_results3.eps', bbox_inches='tight', format='eps', dpi=1000)


# Variation at between classes

In [ ]:

# Nomination network
all_averaged_nomm = {}
for class_id in input_args['classes']:
    all_averaged_nomm[str(class_id)] = {}
    for i in  ['betweenness','closeness','indegree']:
        temp_res = pd.Series([], dtype = float)
        count_res = 0
        for res_avg in list_results_avg_nomm:
            temp_res = temp_res.add(res_avg[str(class_id)][i],fill_value=0)
            count_res = count_res + 1
        all_averaged_nomm[str(class_id)][i] = temp_res/count_res

# Communication network
all_averaged_comm = {}
for class_id in input_args['classes']:
    all_averaged_comm[str(class_id)] = {}
    for i in  ['betweenness','closeness','indegree']:
        temp_res = pd.Series([], dtype = float)
        count_res = 0
        for res_avg in list_results_avg_comm:
            temp_res = temp_res.add(res_avg[str(class_id)][i],fill_value=0)
            count_res = count_res + 1
        all_averaged_comm[str(class_id)][i] = temp_res/count_res

In [ ]:

# Nomination network
class_indegree_nom = []
class_betweenness_nom = []
class_closeness_nom = []

for c in input_args['classes']:
    # per run
    temp1 = all_averaged_nomm[str(c)]
    class_indegree_nom.append(temp1['indegree'])
    class_betweenness_nom.append(temp1['betweenness'])
    class_closeness_nom.append(temp1['closeness'])

# Communication network
class_indegree_com = []
class_betweenness_com = []
class_closeness_com = []

for c in input_args['classes']:
    # per run
    temp1 = all_averaged_comm[str(c)]
    class_indegree_com.append(temp1['indegree'])
    class_betweenness_com.append(temp1['betweenness'])
    class_closeness_com.append(temp1['closeness'])

In [ ]:
# Comunication network
pd_class_indegree_nom = pd.concat(class_indegree_nom, axis=1, keys=[s.name for s in class_indegree_nom])
pd_class_betweenness_nom = pd.concat(class_betweenness_nom, axis=1, keys=[s.name for s in class_betweenness_nom])
pd_class_closeness_nom = pd.concat(class_closeness_nom, axis=1, keys=[s.name for s in class_closeness_nom])

In [ ]:
# Comunication network
pd_class_indegree_com = pd.concat(class_indegree_com, axis=1, keys=[s.name for s in class_indegree_com])
pd_class_betweenness_com = pd.concat(class_betweenness_com, axis=1, keys=[s.name for s in class_betweenness_com])
pd_class_closeness_com = pd.concat(class_closeness_com, axis=1, keys=[s.name for s in class_closeness_com])



In [ ]:
nom_clvar = {'In-degree': (pd_class_indegree_nom.iloc[699]/pd_class_indegree_nom.iloc[0] -1) *100,
           'Betweenness': (pd_class_betweenness_nom.iloc[699]/pd_class_betweenness_nom.iloc[0] -1) *100,
           'Closeness': (pd_class_closeness_nom.iloc[699]/pd_class_closeness_nom.iloc[0] -1) *100}

com_clvar = {'In-degree': (pd_class_indegree_com.iloc[699]/pd_class_indegree_com.iloc[0] -1) *100,
           'Betweenness': (pd_class_betweenness_com.iloc[699]/pd_class_betweenness_com.iloc[0] -1) *100,
           'Closeness': (pd_class_closeness_com.iloc[699]/pd_class_closeness_com.iloc[0] -1) *100}



In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

ax1.set_title('Nomination network')
ax1.set_ylabel('Impact on phyisical activity level (%)')
ax1.set_ylim([0, 18])
ax1.boxplot(nom_clvar.values())
ax1.set_xticklabels(nom_clvar.keys())


ax2.set_title('Communication network')
ax2.set_ylabel('Impact on phyisical activity level (%)')
ax2.set_ylim([0, 18])
ax2.boxplot(com_clvar.values())
ax2.set_xticklabels(com_clvar.keys())
#ax2.legend(['In-degree', 'Betweenness', 'Closeness'], loc='lower right', title="Network interventions")


fig.savefig('../output/sim_results4.eps', bbox_inches='tight', format='eps', dpi=1000)



In [ ]:
i_nom_bp = (pd_class_indegree_nom.iloc[699]/pd_class_indegree_nom.iloc[0] -1) *100
b_nom_bp = (pd_class_betweenness_nom.iloc[699]/pd_class_betweenness_nom.iloc[0] -1) *100
c_nom_bp =  (pd_class_closeness_nom.iloc[699]/pd_class_closeness_nom.iloc[0] -1) *100

i_com_bp = (pd_class_indegree_com.iloc[699]/pd_class_indegree_com.iloc[0] -1) *100
b_com_bp = (pd_class_betweenness_com.iloc[699]/pd_class_betweenness_com.iloc[0] -1) *100
c_com_bp = (pd_class_closeness_com.iloc[699]/pd_class_closeness_com.iloc[0] -1) *100

In [ ]:
com_clvar


In [ ]:
nom_clvar